# DEVELOPMENT NOTEBOOK
---
This notebook is for development phase using python

## A. MINIO CLIENT
----
Create class to interact with minio instance

In [6]:
from loguru import logger
from minio import Minio
from minio.error import S3Error
import urllib3

urllib3.disable_warnings()

class MinioClient():
    """Class to interact with minio
    """
    def __init__(self,
                 config: dict) -> None:
        """Initialize class.
        Args:
            config (dict): minio configuration
        """
        logger.info("INITIALIZING MINIO CLASS")
        self.config = config
        self.client = Minio(
            endpoint = self.config['URL'],
            access_key = self.config['ACCESS_KEY'],
            secret_key = self.config['SECRET_KEY'],
            secure = self.config['TLS'],
            cert_check = (not self.config['TLS']))
        
    def _error(self, function_name: str, message: str, e: Exception) -> None:
        """error wrapper function
        Args:
            function (str): function name
        """
        logger.error(f"AN ERROR OCCURED IN FUNCTION: {function_name}; MESSAGE: {message}; EXCEPTION: {e}")
        return None

    def check_connection(self) -> bool:
        """Check connection to minio server.
        Returns:
            bool: status of the connection
        """
        try:
            self.client.list_buckets()
            logger.info("Connection to minio server is succesful")
            return True
        except S3Error as e:
            self._error("check_connection","CONNECTION TO MINIO SERVER FAILED",e)
            return False
    
    def create_bucket(self, bucket_name: str) -> None:
        """Create bucket inside Minio.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            if not self.client.bucket_exists(bucket_name):
                self.client.make_bucket(bucket_name)
                logger.success(f"Bucket '{bucket_name}' created succesfully")
            else:
                logger.info(f"Bucket {bucket_name} already exists")
        except S3Error as e:
            self._error("create_bucket","CREATING BUCKET FAIL",e)
        
    def delete_bucket(self, bucket_name: str) -> None:
        """Delete bucket inside Minio.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            if self.client.bucket_exists(bucket_name):
                if self.client.remove_bucket(bucket_name):
                    logger.success(f"Bucket {bucket_name} deleted")
            else:
                logger.info(f"Bucket {bucket_name} does not exists")
        except S3Error as e:
            self._error("delete_bucket","BUCKET DELETION FAILED",e)

    def upload_file(self, bucket_name: str, file_path: str, object_name: str) -> None:
        """Upload file to a bucket.
        Args:
            bucket_name (str): minio bucket name
            file_path (str): local file path
            object_name (str): object name in minio bucket
        """
        try:
            self.client.fput_object(bucket_name, object_name, file_path)
            logger.success(f"File '{file_path}' uploaded as '{object_name}' to bucket '{bucket_name}'")
        except S3Error as e:
            self._error("upload_file", "FAILED TO UPLOAD FILE",e)
    
    def download_file(self, bucket_name: str, file_path: str, object_name: str) -> None:
        """Download a file from a bucket.
        Args:
            bucket_name (str): minio bucket name
            file_path (str): local file path
            object_name (str): object name in minio bucket
        """
        try:
            self.client.fget_object(bucket_name, object_name, file_path)
            logger.success(f"File '{object_name}' downloaded to '{file_path}'")
        except S3Error as e:
            self._error("download_file","FAILED TO DOWNLOAD FILE",e)
    
    def list_files(self, bucket_name: str):
        """List all files in a bucket.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            objects = self.client.list_objects(bucket_name)
            for obj in objects:
                logger.info(f"Found object: {obj.object_name}")
        except S3Error as e:
            self._error("list_files","FAILED TO LIST OBJECTS",e)

    def delete_file(self, bucket_name: str, object_name: str):
        """Delete a file from a bucket.
        Args:
            bucket_name (str): minio bucket name
            object_name (str): object name in minio bucket
        """
        try:
            self.client.remove_object(bucket_name, object_name)
            logger.success(f"File '{object_name}' deleted from bucket '{bucket_name}'")
        except S3Error as e:
            self._error("delete_file",f"FAILED TO DELETE FILE {object_name}",e)

    def file_exists(self, bucket_name: str, object_name: str):
        """Check if file exists in bucket
        Args:
            bucket_name (str): minio bucket name
            object_name (str): object_name in bucket
        """
        try:
            self.client.stat_object(bucket_name, object_name)
            logger.info(f"File '{object_name} exists in bucket {bucket_name}")
            return True
        except S3Error as e:
            if e.code == 'NoSuchKey':
                logger.info(f"File '{object_name} does not exists in bucket '{bucket_name}")
                return False
            else:
                self._error("check_file","ERROR CHECKING FILE EXISTENCE",e)
                return False

{'MINIO': {'ACCESS_KEY': 'BQZB2j3nbhPRMeFniKYp', 'SECRET_KEY': 'YDJxEDxZxf8ZdKLaPMImXmDSZoN4kTAv3KynipY8', 'MINIO_URL': 'host.docker.internal:9000', 'MINIO_TLS': True}, 'POSTGRES': {'USER': 'postgres-mlops', 'PASSWORD': 'postgres-123', 'HOST': 'localhost', 'PORT': 5433, 'DATABASE': 'mlops', 'MLFLOW_DATABASE': 'mlflow_backend'}}


In [7]:
from loguru import logger
from minio import Minio
from minio.error import S3Error
import urllib3

urllib3.disable_warnings()

class MinioClient():
    """Class to interact with minio
    """
    def __init__(self,
                 config: dict) -> None:
        """Initialize class.
        Args:
            config (dict): minio configuration
        """
        logger.info("INITIALIZING MINIO CLASS")
        self.config = config
        self.client = Minio(
            endpoint = self.config['MINIO_URL'],
            access_key = self.config['ACCESS_KEY'],
            secret_key = self.config['SECRET_KEY'],
            secure = self.config['MINIO_TLS'],
            cert_check = (not self.config['MINIO_TLS']))
        
    def _error(self, function_name: str, message: str, e: Exception) -> None:
        """error wrapper function
        Args:
            function (str): function name
        """
        logger.error(f"AN ERROR OCCURED IN FUNCTION: {function_name}; MESSAGE: {message}; EXCEPTION: {e}")
        return None

    def check_connection(self) -> bool:
        """Check connection to minio server.
        Returns:
            bool: status of the connection
        """
        try:
            self.client.list_buckets()
            logger.info("Connection to minio server is succesful")
            return True
        except S3Error as e:
            self._error("check_connection","CONNECTION TO MINIO SERVER FAILED",e)
            return False
    
    def create_bucket(self, bucket_name: str) -> None:
        """Create bucket inside Minio.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            if not self.client.bucket_exists(bucket_name):
                self.client.make_bucket(bucket_name)
                logger.success(f"Bucket {bucket_name} created succesfully")
            else:
                logger.info(f"Bucket {bucket_name} already exists")
        except S3Error as e:
            self._error("create_bucket","CREATING BUCKET FAIL",e)
        
    def delete_bucket(self, bucket_name: str) -> None:
        """Delete bucket inside Minio.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            if self.client.bucket_exists(bucket_name):
                if self.client.delete_bucket(bucket_name):
                    logger.success(f"Bucket {bucket_name} deleted")
            else:
                logger.info(f"Bucket {bucket_name} does not exists")
        except S3Error as e:
            self._error("delete_bucket","BUCKET DELETION FAILED",e)

    def upload_file(self, bucket_name: str, file_path: str, object_name: str) -> None:
        """Upload file to a bucket.
        Args:
            bucket_name (str): minio bucket name
            file_path (str): local file path
            object_name (str): object name in minio bucket
        """
        try:
            self.client.fput_object(bucket_name, object_name, file_path)
            logger.success(f"File '{file_path}' uploaded as '{object_name}' to bucket '{bucket_name}'")
        except S3Error as e:
            self._error("upload_file", "FAILED TO UPLOAD FILE",e)
    
    def download_file(self, bucket_name: str, file_path: str, object_name: str) -> None:
        """Download a file from a bucket.
        Args:
            bucket_name (str): minio bucket name
            file_path (str): local file path
            object_name (str): object name in minio bucket
        """
        try:
            self.client.fget_object(bucket_name, object_name, file_path)
            logger.success(f"File '{object_name}' downloaded to '{file_path}'")
        except S3Error as e:
            self._error("download_file","FAILED TO DOWNLOAD FILE",e)
    
    def list_files(self, bucket_name: str):
        """List all files in a bucket.
        Args:
            bucket_name (str): minio bucket name
        """
        try:
            objects = self.client.list_objects(bucket_name)
            for obj in objects:
                logger.info(f"Found object: {obj.object_name}")
        except S3Error as e:
            self._error("list_files","FAILED TO LIST OBJECTS",e)

    def delete_file(self, bucket_name: str, object_name: str):
        """Delete a file from a bucket.
        Args:
            bucket_name (str): minio bucket name
            object_name (str): object name in minio bucket
        """
        try:
            self.client.remove_object(bucket_name, object_name)
            logger.success(f"File '{object_name}' deleted from bucket '{bucket_name}'")
        except S3Error as e:
            self._error("delete_file",f"FAILED TO DELETE FILE {object_name}",e)


In [4]:
minio = MinioClient(config['MINIO'])

2024-11-01 21:34:49.269 | INFO     | __main__:__init__:17 - INITIALIZING MINIO CLASS


In [5]:
minio.check_connection()

2024-11-01 21:34:51.933 | INFO     | __main__:check_connection:41 - Connection to minio server is succesful


True

In [9]:
minio.create_bucket("aiml-test")

2024-11-01 22:14:21.688 | SUCCESS  | __main__:create_bucket:55 - Bucket aiml-test created succesfully


In [11]:
minio.upload_file("aiml-test","/home/st_yudi/personal-github-repository/aiml-infrastructure/config/config.yaml.example","config/config.yaml.example")

2024-11-01 22:14:57.714 | SUCCESS  | __main__:upload_file:84 - File '/home/st_yudi/personal-github-repository/aiml-infrastructure/config/config.yaml.example' uploaded as 'config/config.yaml.example' to bucket 'aiml-test'


## PYTHON DECORATOR

In [64]:
import time
from loguru import logger
from typing import Callable, List, Optional

def timer(func: Callable) -> Callable:
    """decorator for counting time execution of a 
       function
    Args:
        func (Callable):
            generic function
    Returns:
        Callable:
            wrapped function
    """
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        logger.info(f"Execution time: {execution_time} seconds")
        return result
    return wrapper

def debugger(func: Callable) -> Callable:
    """decorator for debuging function
    Args:
        func (Callable):
            generic function
    Returns:
        Callable:
            wrapped function
    """
    def wrapper(*args, **kwargs):
        logger.info(f"Calling {func.__name__} with args: {args} kwargs: {kwargs}")
        result = func(*args, **kwargs)
        logger.info(f"{func.__name__} returned: {result}")
        return result
    return wrapper

def memoize(func: Callable) -> Callable:
    """decorator to save result of a function
    Args:
        func (Callable):
            generic function
    Returns:
        Callable:
            wrapped function
    """
    cache = {}
    def wrapper(*args):
        if args in cache:
            return cache[args]
        else:
            result = func(*args)
            cache[args] = result
            return result
    return wrapper

def retry(max_attempts: int, delay: int = 1) -> Callable:
    """decorator for retrying function in case failed
    Args:
        max_attempts (int):
            max_attempts of a generic function
        delay (int, optional):
            number of second function call delayed. Defaults to 1.
        func (Callable):
            generic function
    Returns:
        Callable:
            wrapped function
    """
    def decorator(func: Callable) -> Callable:
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    logger.error(f"Attempt {attempts} failed: {e}")
                    time.sleep(delay)
            logger.error(f"Function failed after {max_attempts} attempts")
        return wrapper
    return decorator

def exception_handler(func: Callable) -> Callable:
    """decorator for handling error
    Args:
        func (Callable):
            generic function
    Returns:
        Callable:
            wrapped function
    """
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logger.error(f"An exception occured: {str(e)}")
            raise e
    return wrapper

In [65]:
@exception_handler
def divide(x, y):
    result = x / y
    return result
divide(10, 0)  
# Output: An exception occurred: division by zero
# Retries 3 times with a 2-second delay between attempts

2024-09-23 19:46:06.847 | ERROR    | __main__:wrapper:99 - An exception occured: division by zero


ZeroDivisionError: division by zero

In [43]:
s = "axc"
t = "ahbgdc"

def isSubsequence(s, t):
    
    """
    :type s: str
    :type t: str
    :rtype: bool
    """
    i, j = 0, 0
    while i < len(s) and j < len(t):
    # If characters match, move the pointer of s forward
        if s[i] == t[j]:
            i += 1
    # Always move the pointer of t forward
        j += 1

    # If we have traversed all characters of s, return True
    return i == len(s)

isSubsequence(s,t)

False

True